# Radioactive Decay Energy

Within the ejecta of a supernova, the $\gamma$-rays largely come from the decay of $^{56}Ni$ into $^{56}Co$, which releases a significant amount of energy. 

When $^{56}Ni$ decays into $^{56}Co$ it can release a $\gamma$-ray at several different transition levels. Each transition level has an energy and an associated probability out of 100 decays. For example, the transition from Energy level 9 to Energy level 7 has an energy of 0.270 Mev and a probability of 36.5 out of 100 decays. To find the total energy per decay you multipliy each energy with its associated probability and add them all up.

In [1]:
import numpy as np
import pandas as pd
from astropy import units as u
from astropy import constants as const

from tardis.energy_input.gamma_ray_channel import create_isotope_dicts
from tardis.energy_input.gamma_ray_channel import create_inventories_dict
from tardis.energy_input.gamma_ray_channel import calculate_total_decays
from tardis.energy_input.gamma_ray_channel import create_isotope_decay_df

/home/ceceliapowers/software/tardis/tardis/__init__.py:23: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
# energies of each transition
t_energies = np.array([0.270, 0.750, 0.480, 1.56, 0.812, 0.158]) * u.MeV
# probabilities of each transition
t_prob = np.array([.365, .495, .366, .140, .860, 1.00])

energy_per_decay = sum(t_energies * t_prob)
energy_per_decay

<Quantity 1.7202 MeV>

From the above cell, we get the energy per transition of 1.72 MeV. Note that this comes from a simplified scheme of energies and the real total energy per $^{56}Ni$ decay we use is 1.75 MeV. <strong data-cite="1994Nadyozhin">[]</strong> 

The $^{56}Co$ produced from the decay of $^{56}Ni$ is also radioactive and will decay into $^{56}Fe$ and release more $\gamma$-rays, however this decay is more complicated than the decay of $^{56}Ni$. Whereas $^{56}Ni$ only decays through electron capture, $^{56}Co$ can decay either by electron capture, which occurs for 81 out of 100 cases, or through positron decay, which occurs for 19 out of 100 cases.

Positron decay produces positrons with a given kinetic energy, that will eventually annihilate with electrons to produce two 0.511 MeV $\gamma$-rays. The scheme of decays for $^{56}Co$ is slightly more complicated than the $^{56}Ni$ scheme, but to find the total energy per decay, you follow the same process. The total energy per decay from $\gamma$-rays is 3.61 MeV and the total kinetic energy of positrons is 0.12 MeV

 The total rate of energy production for a mass of $^{56}Ni$ at a given time is given by the following equation:

$$\epsilon = \frac{M_\odot}{56m_{u}}\frac{1}{\tau_{\text{Co}}-\tau_{\text{Ni}}}[[Q_{\text{Ni}}(\frac{\tau_{\text{Co}}}{\tau_{\text{Ni}}}-1)-Q_{\text{Co}}]\exp(-t/\tau_{\text{Ni}})+Q_{\text{Co}}\exp(-t/\tau_{\text{Co}})]\frac{M_{Ni0}}{M_\odot}$$

$M_\odot$ is a solar mass. $56_{u}$ is 56 atomic mass units. 

$\tau_{Ni}$ is the lifetime of $^{56}Ni$ which is 8.80 days and $\tau_{Co}$ is the lifetime of $^{56}Co$ which is 111.3 days. 

$Q_{\text{Ni}}$ is the energy per decay of $^{56}Ni$ which is 1.75 MeV and $Q_{\text{Co}}$ is the sum of the energy per decay pf $^{56}Co$ from $\gamma$-rays and the kinetic energy from positrons which is 3.73 MeV

If we plug these values into the equation we get the equation:

$$\epsilon = (6.45e43\exp(-t/8.8)+1.45e43\exp(-t/111.3))\frac{M_{Ni0}}{M_\odot} erg/s$$

In [3]:
#time in days
time = 10 * u.day
#mass of Ni56 in solar masses
m_Ni56 = 1 * const.M_sun

energy_production_rate = (6.45e43 * np.exp(-time/(8.8*u.day)) + 1.45e43 * np.exp(-time/(111.3*u.day))) * (m_Ni56/ const.M_sun) * u.erg /u.s

print(f"The total energy production rate for 1 solar mass of 56Ni after 10 days is: {energy_production_rate:.2e}")

The total energy production rate for 1 solar mass of 56Ni after 10 days is: 3.40e+43 erg / s


In [4]:
total_energy_production = 1.885e50 * (m_Ni56/const.M_sun) * u.erg
print(f"The total energy production for 1 solar mass of 56Ni is: {total_energy_production}")

The total energy production for 1 solar mass of 56Ni is: 1.885e+50 erg


Next we show how we can find the same result from functions in TARDIS-HE

In [5]:
mass_fractions = {0:1,
                  1:1,
                  2:1,
                  3:1,
                  4:1}

raw_isotope_abundance = pd.DataFrame(mass_fractions, index=pd.MultiIndex.from_tuples([(28, 56)], names=["atomic_number", "mass_number"]))
raw_isotope_abundance

,,0,1,2,3,4
atomic_number,mass_number,,,,,
28,56,1,1,1,1,1


In [6]:
shell_masses = np.ones(5)*0.2*const.M_sun.to(u.g)
shell_masses

<Quantity [3.97681974e+32, 3.97681974e+32, 3.97681974e+32, 3.97681974e+32,
           3.97681974e+32] g>

In [7]:
isotope_dict = create_isotope_dicts(raw_isotope_abundance, shell_masses)
isotope_dict

{0: {'Ni56': 3.976819741396102e+32},
 1: {'Ni56': 3.976819741396102e+32},
 2: {'Ni56': 3.976819741396102e+32},
 3: {'Ni56': 3.976819741396102e+32},
 4: {'Ni56': 3.976819741396102e+32}}

In [8]:
inventories = create_inventories_dict(isotope_dict)
inventories

{0: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020,
 1: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020,
 2: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020,
 3: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020,
 4: Inventory activities (Bq): {'Ni-56': 5.653446204017355e+48}, decay dataset: icrp107_ame2020_nubase2020}

In [10]:
cumulative_decay_df = calculate_total_decays(inventories, time)
print(cumulative_decay_df)

                      number_of_decays
shell_number isotope                  
0            Ni56         2.913217e+54
             Co56         1.501419e+53
1            Ni56         2.913217e+54
             Co56         1.501419e+53
2            Ni56         2.913217e+54
             Co56         1.501419e+53
3            Ni56         2.913217e+54
             Co56         1.501419e+53
4            Ni56         2.913217e+54
             Co56         1.501419e+53


In [23]:
total_Ni56_decays = 0
for i in range(5):
    total_Ni56_decays += cumulative_decay_df.loc[(i,"Ni56")].item()

total_Co56_decays = 0
for i in range(5):
    total_Co56_decays += cumulative_decay_df.loc[(i,"Co56")].item()

In [24]:
Ni56_decay_energy = total_Ni56_decays * 1.75*u.MeV
Co56_decay_energy = total_Co56_decays * 3.61*u.MeV

In [26]:
total_decay_energy = Ni56_decay_energy + Co56_decay_energy
total_decay_energy.to(u.erg)

<Quantity 4.51825242e+49 erg>